In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
torch.manual_seed(1234)

#build model
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        """
        Encoder
        """
        self.conv1c = nn.Conv1d(1, 6, kernel_size=3,stride=1,padding=1)
        self.bt = nn.BatchNorm1d(6)
        self.relu = nn.ReLU(inplace=True)
        self.mp1 = nn.MaxPool1d(kernel_size=2,stride=2,return_indices=True)
        """
        Decoder
        """
        self.mup1 = nn.MaxUnpool1d(kernel_size=2,stride=2)
        self.conv1d = nn.Conv1d(6, 6, kernel_size=3,padding=1)
        self.conv11d = nn.Conv1d(6, 3, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(12, 3)
        

    def forward(self, x):
        x=self.conv1c(x)
        x=self.bt(x)
        x = self.relu(x)
        x,idx2 = self.mp1(x)    
        x1=x
        x=self.mup1(x,idx2)
        x=self.conv1d(x)
        x=self.bt(x)
        x=self.relu(x)
        x=self.conv11d(x)
        x=x.view(-1,self.num_flat_features(x))
        return x
    def num_flat_features(self,x):
        size=x.size()[1:]
        num_features=1
        for s in size:
            num_features *= s
        return num_features
net = Net()


In [ ]:
%matplotlib inline
#print(__doc__)
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split


# import some data to play with
iris = datasets.load_iris()
Xi = iris.data
yi = iris.target
x_train, x_test, y_train, y_test = train_test_split(Xi, yi, test_size=.5)


In [ ]:

#hyperparameters
lr = 0.01

#choose optimizer and loss function
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=lr)
m=len(x_train)
x_train.shape=(m,1,4)
x_train = Variable(torch.Tensor(x_train).float())
Y = Variable(torch.Tensor(y_train).long())
    
#train
for epoch in range(1000):
    #Y=Y.view(1,1,150,-1)
    #feedforwad - backprop
    optimizer.zero_grad()
    out = net(x_train)
    loss = criterion(out, Y)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print('number of epoch', epoch, 'loss', loss.data)


In [ ]:
#get prediction 
x_test=np.array(x_test) 
m=len(x_test) 
x_test.shape=(m,1,4) 
X = Variable(torch.Tensor(x_test).float()) 
Y = Variable(torch.Tensor(y_test).long())
out = net(X)
_, predicted = torch.max(out.data, 1) 
print(predicted)

#get accuration 
print('Accuracy of the network %d %%' % (100 * torch.sum(Y==predicted) / m))


In [ ]:
import matplotlib
out=out.data.numpy()
x=out[:,2]
y=out[:,3]
colors=['red','green','blue']
plt.scatter(x,y,c=y_test,cmap=matplotlib.colors.ListedColormap(colors))
plt.show()